In [ ]:
################This python script takes an excel sheet containing the number of predicted LULC points/pixels inside###################### 
################each ground truth (GT) polygon. The script then calculates the percentage of correctly and incorrectly#################### 
################classified pixels per GT polygon to work out the total accuracy per GT polygon############################################
################The excel data is generated using the Summarise statistics tool in ArcGIS Pro. Inputs are:################################
################the RastertoPoint_SpatialJoin layer and the summary statistics table###########################

##############Import the necessary python libraries##################
import pandas as pd

##############Enable Pandas to display all rows and columns of a Dataframe################################################################
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

###############Define the file path to your excel document and the sheet containing the data#############

file_path = r'C:\Users\MAPWAPS\Desktop\Polygons\Accuracy_per_polygon_uMzim.xlsx'

sheet_name = 'Sheet1'  # Replace with another sheet name if different e.g 'Sheet3'

############Read the data into a DataFrame and visualize it#####################################
df = pd.read_excel(file_path, sheet_name=sheet_name)
df


############Count the total number of points per duplicate ID (polygon) by grouping the points by 'ID' and visualize the results#####################

summed_df = df.groupby('JOIN_FID', as_index=False)['FREQUENCY'].sum()######################
summed_df

##########Merge summed data back to the original dataframe and store the sum per ID in a new field called FREQUENCY_Total####################

df = df.merge(summed_df, on='JOIN_FID', suffixes=('', '_Total'))
df


###########Add a field called Percentage to the original dataframe and calculate the percentage of each individual######
###########ID  by dividing the count of each duplicate ID by the sum of each grouped ID####################################

df['Percentage'] = (df['FREQUENCY'] / df['FREQUENCY_Total']) * 100
df


###########Add a field called Correct that identifies the correctly aclassified IDs by comparing#################################
###########the LULC ID field from the GT data and the grid_code field from the MAPWAPS classification #########################################

df['Correct'] = df['grid_code'] == df['LULC_ID']
df


#############Create a dataframe called sum_Correct that stores the sum of all correctly classified IDs ############################
#############Note that IDs with zero correctly classified pixels will be missinng as they are assigned are assigned NaN values######################################

sum_Correct = df[df['Correct']].groupby('JOIN_FID')['FREQUENCY'].sum().reset_index(name='Correct_pixels')
sum_Correct


##############Merge the data frames which we used to calcualte the total for each ID################################
##############and the one we used to store the sum for each correctly classified ID. Note that IDs with zero###########################
##############correctly classified will be filled with zero in accuracy_df veacuse of the code: .fillna(0)###########################################################

accuracy_df = pd.merge(summed_df, sum_Correct, on='JOIN_FID', how='left').fillna(0)
accuracy_df



##############Calculate the accuracy for each ID by dividing the sum of correctly classified pixels for each ID #########################################
##############by the sum of all pixels per ID and multiplying by 100. Fill Nan values with 0 to account for IDs with#####################################
##############zero correctly classified pixels##########################################################################################################

accuracy_df['Accuracy'] = (accuracy_df['Correct_pixels'].fillna(0) / accuracy_df['FREQUENCY']) * 100
accuracy_df


########Extract the LULC name and LULC ID from the GT data and ensure that there no duplicat IDs match the accuracy resulst##########################################################################################
LULC_class= df.groupby('JOIN_FID', as_index=False)['LULC'].max()
LULC_ID= df.groupby('JOIN_FID', as_index=False)['LULC_ID'].max()

LULC_df = pd.merge(LULC_class,LULC_ID, on='JOIN_FID', how='inner')

################ Join the dataframe with the accuracy results with original dataframe#########################################################################

joined_df = pd.merge(LULC_df,accuracy_df, on='JOIN_FID', how='inner').rename(columns={'FREQUENCY': 'Number of pixels per GT polygon'})
joined_df

##########Export the results to an Excel file####################################################################

joined_df.to_excel('Final_GT_Results_Umzimvubu.xlsx', index=False)  



